# Анализ базы данных книг

**Цель проекта** — проанализировать базу данных. 

В ней содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
# сохраняем коннектор

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `books`Содержит данные о книгах:**

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [2]:
# Посмотрим на первые пять строк таблицы authors 
query = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `authors` содержит данные об авторах:**

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [3]:
# Посмотрим на первые пять строк таблицы publishers 
query = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `publishers` содержит данные об издательствах:**

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [4]:
# Посмотрим на первые пять строк таблицы ratings 
query = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `ratings` содержит данные о пользовательских оценках книг:**

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [5]:
# Посмотрим на первые пять строк таблицы reviews 
query = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица `reviews` содержит данные о пользовательских обзорах на книги:**

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Количество книг, вышедших после 1 января 2000 года

In [6]:
query = '''SELECT COUNT (book_id)
           FROM books
           WHERE publication_date > '2000-01-01'
           '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


* После 1 января 2000 года вышло 819 книг.

## Количество обзоров и средняя оценка каждой книги

In [7]:
query = '''SELECT b.book_id, title, COUNT (DISTINCT r.review_id) AS reviews, AVG(rat.rating) AS middle_rating
           FROM books AS b
           LEFT JOIN reviews AS r ON b.book_id = r.book_id
           LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
           GROUP BY b.book_id 
          '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,reviews,middle_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


In [8]:
query = '''SELECT b.book_id, title, COUNT (DISTINCT r.review_id) AS reviews, AVG(rat.rating) AS middle_rating
           FROM books AS b
           LEFT JOIN reviews AS r ON b.book_id = r.book_id
           LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
           GROUP BY b.book_id
           ORDER BY reviews DESC,middle_rating DESC
           LIMIT 10
          '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,reviews,middle_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


* **Самые популярные по обзорам книги:** 

* "Сумерки (первая часть)" - 7 (рейтинг 3,7)
- "Гарри Поттер и узник Азкабана" - 6 (рейтинг 4,4)
- "Гарри Поттер и тайная комната" - 6 рецензий (рейтинг 4,3).

In [9]:
query = '''SELECT b.book_id, title, COUNT (DISTINCT r.review_id) AS reviews, AVG(rat.rating) AS middle_rating
           FROM books AS b
           LEFT JOIN reviews AS r ON b.book_id = r.book_id
           LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
           GROUP BY b.book_id
           ORDER BY middle_rating DESC, reviews DESC
           LIMIT 10
          '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,reviews,middle_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.0
1,553,School's Out—Forever (Maximum Ride #2),3,5.0
2,444,Moneyball: The Art of Winning an Unfair Game,3,5.0
3,169,Crucial Conversations: Tools for Talking When ...,2,5.0
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.0
5,62,Alas Babylon,2,5.0
6,57,Act of Treason (Mitch Rapp #9),2,5.0
7,20,A Fistful of Charms (The Hollows #4),2,5.0
8,55,A Woman of Substance (Emma Harte Saga #1),2,5.0
9,76,Angels Fall,2,5.0


* **Среди книг с наивысшим баллом 5.0 выберем ТОП-3 у которых больше всего рецензий:**


- "Грязная работа" (4)

-  "Максимальная поездка: школа закрыта навсегда" (3)

-  "MoneyBall. Как математика изменила самую популярную спортивную лигу в мире" (3)


## Издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [10]:
query = '''SELECT  p.publisher, COUNT(b.book_id) AS count_books
           FROM (SELECT*
                 FROM books 
                 WHERE num_pages > 50) AS b
           LEFT JOIN publishers AS p ON b.publisher_id  = p.publisher_id 
           GROUP BY p.publisher
           ORDER BY count_books DESC
           LIMIT 5
           '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


* Издательство, которое выпустило наибольшее число книг толще 50 страниц - "Penguin Books", которое выпустило 42 таких книги.

## Автор с самой высокой средней оценкой книг среди книг с 50 и более оценками

In [11]:
query = '''SELECT a.author_id, a. author , AVG (avg_rating) as avg_rating
           FROM (SELECT book_id, COUNT (rating) AS count_rating, AVG (rating) AS avg_rating
                       FROM ratings 
                       GROUP BY book_id
                       HAVING COUNT (rating) >= 50) AS r
           LEFT JOIN books AS b ON r.book_id = b.book_id
           LEFT JOIN authors AS a ON b.author_id = a.author_id
           GROUP BY a.author_id
           ORDER BY avg_rating DESC
          '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.258446
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645
5,621,William Golding,3.901408
6,235,J.D. Salinger,3.825581
7,469,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,630,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,106,Dan Brown,3.754540


* Автор с самой высокой средней оценкой 4.28 - Джоан Роулинг с иллюстрациями Мари ГранПре.

## Среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query = ''' SELECT AVG (count_review_id)
            FROM (SELECT u.username, COUNT (rev.review_id) AS count_review_id
                  FROM  (SELECT username, COUNT(rating) AS count_rating
                    FROM ratings 
                    GROUP BY username 
                    HAVING COUNT(rating) > 48) AS u
                 LEFT JOIN reviews AS rev ON u.username = rev.username
                 GROUP BY u.username) AS r
           '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


* Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 обзора.

## Вывод

**По итогу анализа базы данных книг:**

* Проанализировали базу из 1000 книг.


* Выяснили, что после 1 января 2000 года вышло 819 книг.


* Посчитали для каждой книги количество обзоров и среднюю оценку.


* Самые популярные по обзорам книги: "Сумерки (первая часть)" - 7 (рейтинг 3,7), "Гарри Поттер и узник Азкабана" - 6 (рейтинг 4,4), "Гарри Поттер и тайная комната" - 6 рецензий (рейтинг 4,3).


* Среди книг с наивысшим баллом 5.0 выбрали ТОП-3 у которых больше всего рецензий: "Грязная работа" (4), "Максимальная поездка: школа закрыта навсегда" (3), "MoneyBall. Как математика изменила самую популярную спортивную лигу в мире" (3).


* Издательство, которое выпустило наибольшее число книг толще 50 страниц - "Penguin Books", насчитывает 42 таких книги.


* Автор с самой высокой средней оценкой 4.28 - Джоан Роулинг с иллюстрациями Мари ГранПре.


* Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 обзора.